In [ ]:
import socket
from cryptography.hazmat.primitives.asymmetric import padding
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.backends import default_backend
import ssl

def load_private_key():
    with open("private_key.pem", "rb") as key_file:
        return serialization.load_pem_private_key(
            key_file.read(),
            password=None,
            backend=default_backend()
        )

def load_public_key():
    with open("server_public_key.pem", "rb") as key_file:
        return serialization.load_pem_public_key(
            key_file.read(),
            backend=default_backend()
        )

def decrypt_data(private_key, encrypted_data):
    return private_key.decrypt(
        encrypted_data,
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )

def verify_signature(public_key, signature, combined_hash):
    try:
        public_key.verify(
            signature,
            combined_hash,
            padding.PSS(
                mgf=padding.MGF1(hashes.SHA256()),
                salt_length=padding.PSS.MAX_LENGTH
            ),
            hashes.SHA256()
        )
        return True
    except Exception as e:
        print(f"Verification failed: {e}")
        return False

def start_server():
    private_key = load_private_key()
    public_key = load_public_key()
    context = ssl.create_default_context(ssl.Purpose.CLIENT_AUTH)
    context.load_cert_chain(certfile='server.crt', keyfile='server.key')
    server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server_socket.bind(('localhost', 65432))
    server_socket.listen(5)
    secure_socket = context.wrap_socket(server_socket, server_side=True)

    try:
        while True:
            client_socket, addr = secure_socket.accept()
            print(f"Connected securely by {addr}")

            data = b""
            while True:
                part = client_socket.recv(1024)
                if part:
                    data += part
                    print(f"Received part ({len(part)} bytes):", part)
                else:
                    print("No more data from client.")
                    break

            if data:
                print(f"Total received data ({len(data)} bytes):", data)
                if len(data) == 768:  # 确保接收到完整的数据
                    encrypted_order_info = data[:256]
                    encrypted_payment_info = data[256:512]
                    signature = data[512:768]

                    try:
                        order_info = decrypt_data(private_key, encrypted_order_info)
                        payment_info = decrypt_data(private_key, encrypted_payment_info)

                        print("Decrypted Order Info:", order_info)
                        print("Decrypted Payment Info:", payment_info)

                        combined_hash = order_info + payment_info
                        if verify_signature(public_key, signature, combined_hash):
                            print("Signature verification successful.")
                        else:
                            print("Signature verification failed.")
                    except Exception as e:
                        print(f"Error processing data: {e}")
                else:
                    print("Data incomplete or corrupt.")
            client_socket.shutdown(socket.SHUT_RDWR)
            client_socket.close()

    except Exception as e:
        print(f"Server error: {e}")
    finally:
        secure_socket.close()

if __name__ == "__main__":
    start_server()



Connected securely by ('127.0.0.1', 63255)
Received part (768 bytes): b'\xb8\x81R%w \xf2D\x1c\xb1\xb0\x97[\xbc\xcb\x83\xbc\x9eJ\xe9%Y%\x1e\x9b\x8c\xe5\x0b\xea\xcaVhG\x1c\xca\xe9\xf9m{\x89Q1\x99>\xa40\x8cv\x9cp\xd7\xc4\xc6\x90\xe8\xad\xfeJ\xccuH\x10e4\x06z\x0f\x1f\xb4\xf7!\xa1|\xc9\xc3\xec\xfb\x86\x018gT;e<\xcc\xe6\xe5\x06Z\xb3\xfc\xc1\xf9!\xe3\xbc$\xdf-MU\x18\x0fgK\xb0\xd8\x0e2c\x9dI\x16}\x84\x06\xce]\xd4\xbc\x13\xba\xf8\xc0-\xa4\xedZx\xfe\x0f\x9dt*\x80\xee!,\x99Z,\x97\xd3\x1f\x86\x82\xe6\xecW\x03\xf1B(\x05S/K\x8d\xb4\xd7\xd7\x8f\xd0M\x9d,I\x9c\xab\xb1^\x1e\xec\x1f8A\x08\x89\x9c\x15J\xa3-\xb1t\xde\xd7!\x84\x90F\xc2\x7f7\xcf\x17h\xcd\x01\xeb\xda\xc95\xb7\xdf\xb9[#CE\x19\x0e\x8c\xca{\x93\xa59\xa8\t\xe7-<]\x05\xf1y\xba\x84e\x9c\xa3\xaa\x91"\xef\x8aJon\xe3\xaf\x17j\xb1/\xc5\'\x0c\x88\xed-\xd6\xcd\x8dE\xa7R:\xd0\t\xb1\x028:\xea\xe3\xa1\xbb\x8c:D\xed#;\xd4L\x87\x1eA\xa1\xf9}\x97\r\xce\xcf\xf0\xa3g>\xc2\xb3j\x97x\xf5l1[\xf4\xfa\xfd\xb6\x0b\x86\xaaG\x99<H\xcb\x04\xd2\xf3\xf0u\xfa\xc6\x07\x80\x